In [2]:
import pathlib
import os

import h5py
import pandas as pd
import json
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import dask.array as da
import dask.dataframe as dd
import skimage 
from glob import glob

import cloudpickle
import sklearn
from sklearn.linear_model import LinearRegression as LinReg


import SessionTools.two_photon as st2p


%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [3]:

def build_dict(date, fly, sess, sess_dir,
               genotype = "ER4d_sytjGCaMP7f_EPG_jRGECO1a",
               basedir = pathlib.Path('/media/mplitt/SSD_storage1/2PData')):
    basedir = pathlib.Path(basedir.joinpath(genotype))
    
    basename_input = pathlib.Path(sess_dir.joinpath(sess))
    metadata = st2p.preprocessing.bruker_metadata.read(basename_input)

    h5name = f'/media/mplitt/SSD_storage1/2P_scratch/{genotype}/{date}/{fly}/{sess}/data.h5'
    if not os.path.exists(h5name):
        return None
        # tiff_data = st2p.preprocessing.tiff_tools.read(basename_input, 
        #                                             metadata['size'],metadata['layout'], first_chan=1)
        # st2p.preprocessing.tiff_tools.convert_to_hdf5(tiff_data,h5name, overwrite=True)
    napari_outputs_file = f'/media/mplitt/SSD_storage1/2P_scratch/{genotype}/{date}/{fly}/{sess}/napari.pkl'
    if not os.path.exists(napari_outputs_file):
        return None
                
    f = h5py.File(h5name)
    data = f['/data'][:]

    ref_img = st2p.preprocessing.motion_correction.make_ref_img(data,0)
    data_corr, shifts, error, diffphase = st2p.preprocessing.motion_correction.align_data_chunk(data, 
                                                                                                ref_img[0,:,:,:], 
                                                                                                in_place=False)

    napari_outputs_file = f'/media/mplitt/SSD_storage1/2P_scratch/{genotype}/{date}/{fly}/{sess}/napari.pkl'
    if not os.path.exists(napari_outputs_file):
        return None
    
    with open(napari_outputs_file, 'rb') as file:
        np_layers = cloudpickle.load(file)
    print(np_layers.keys())
                    
    masks_EB = np_layers['rois_EB']
    masks_R4d = np_layers['rois_R4d']
    bckgnd = np_layers['background']
                
        
        
    csv_files = glob(f'/media/mplitt/SSD_storage1/2PData/{genotype}/{date}/{fly}/{sess}/*.csv')
    vr_file = pathlib.Path(csv_files[0])
    df = dd.read_csv(vr_file).compute()

    frame_times = np.array(metadata['frame_times']).mean(axis=-1)*1000
    df_aligned = st2p.preprocessing.signals.align_vr_2p(df,frame_times)

    # remove VR artifact
    F_EB, notF_EB = st2p.preprocessing.signals.extract_2p_timeseries(data_corr, masks_EB, 16, bckgnd_mask = bckgnd, max_proj=False) 
    F_R4d, notF_R4d = st2p.preprocessing.signals.extract_2p_timeseries(data_corr, masks_R4d, 16, bckgnd_mask = bckgnd, max_proj=False) 
    
    F_EB_bcorr = 0*F_EB
    F_R4d_bcorr = 0*F_R4d
    for ch in range(F_EB.shape[0]):
        lr = LinReg().fit(notF_EB[ch,np.newaxis, :].T, F_EB[ch,:,:].T)
        F_EB_bcorr[ch,:,:] = F_EB[ch,:,:]-lr.predict(notF_EB[ch,np.newaxis,:].T).T 
        
        lr = LinReg().fit(notF_R4d[ch,np.newaxis, :].T, F_R4d[ch,:,:].T)
        F_R4d_bcorr[ch,:,:] = F_R4d[ch,:,:]-lr.predict(notF_R4d[ch,np.newaxis,:].T).T 
    
    F_EB_bleed = 0*F_EB
    F_R4d_bleed = 0*F_R4d
    lr = LinReg().fit(F_EB_bcorr[1,:, :].T, F_EB_bcorr[0,:,:].T)
    F_EB_bleed[ch,:,:] = F_EB_bcorr[0,:,:]-1.*lr.predict(F_EB_bcorr[1,:,:].T).T 
        
    lr = LinReg().fit(F_R4d_bcorr[0,:, :].T, F_R4d_bcorr[1,:,:].T)
    F_R4d_bleed[ch,:,:] = F_R4d_bcorr[1,:,:]-1,*lr.predict(F_R4d_bcorr[0,:,:].T).T 
    
        
    
    F_EB_sm = sp.ndimage.gaussian_filter1d(F_EB_bleed,2,axis=-1)
    F_EB_sm = sp.ndimage.gaussian_filter1d(F_EB_sm,1.5,axis=1, mode='wrap')
    
    F_R4d_sm = sp.ndimage.gaussian_filter1d(F_R4d_bleed,2,axis=-1)
    F_R4d_sm = sp.ndimage.gaussian_filter1d(F_R4d_sm,1.5,axis=1, mode='wrap')

    dff_EB = sp.stats.zscore(F_EB_sm[0,:,:],axis=-1)
    dff_R4d = sp.stats.zscore(F_R4d_sm[1,:,:],axis=-1)

    heading = np.angle(np.exp(1j*(np.pi-df_aligned[' Heading'].to_numpy().ravel())))

    bin_centers =  np.linspace(-np.pi, np.pi, num=16)[:,np.newaxis]
    dff_EB_complex = dff_EB*np.cos(bin_centers) + 1j*dff_EB*np.sin(bin_centers)
    rho_EB, phi_EB = np.abs(dff_EB_complex.mean(axis=0)), np.angle(dff_EB_complex.mean(axis=0))
    
    dff_R4d_complex = dff_R4d*np.cos(bin_centers) + 1j*dff_R4d*np.sin(bin_centers)
    rho_R4d, phi_R4d = np.abs(dff_R4d_complex.mean(axis=0)), np.angle(dff_R4d_complex.mean(axis=0))

    data = {'fly': fly,
            'date': date,
            'session': sess,
            'F_EB': F_EB,
            'not_F_EB': notF_EB,
            'F_EB_bcorr':F_EB_bcorr,
            'F_EB_bleed': F_EB_bleed,
            'F_EB_sm': F_EB_sm,
            'F_R4d': F_R4d,
            'not_F_R4d': notF_R4d,
            'F_R4d_bcorr':F_R4d_bcorr,
            'F_R4d_bleed': F_R4d_bleed,
            'F_R4d_sm': F_R4d_sm,
            'df_aligned': df_aligned,
            'dff_EB': dff_EB,
            'dff_R4d': dff_R4d,
            'heading': heading,
            'bin_centers': bin_centers,
            'dff_EB_complex': dff_EB_complex,
            'dff_R4d_complex': dff_R4d_complex,
            'rho_EB': rho_EB,
            'phi_EB': phi_EB,
            'rho_R4d': rho_R4d,
            'phi_R4d': phi_R4d}
    
    return data

    
               

In [5]:
genotype = "ER4d_sytjGCaMP7f_EPG_jRGECO1a"
basedir = pathlib.Path(f'/media/mplitt/SSD_storage1/2PData/{genotype}')

for date_dir in basedir.glob('*'):
    date = date_dir.stem
    for fly_dir in date_dir.glob('fly*'): 
        fly = fly_dir.stem
        
        for sess_dir in fly_dir.glob("*"):
            if sess_dir.is_dir() and "SingleImage" not in sess_dir.stem:
                sess = sess_dir.stem
                
                sess_pkl_dir = f'/media/mplitt/SSD_storage1/2P_scratch/{genotype}/{date}/{fly}/{sess}/data.pkl'
                if not os.path.exists(sess_pkl_dir):
                    # print(date, fly, sess)
                    try:
                        data = build_dict(date, fly, sess, sess_dir)

                        if data is not None:
                            try:
                                with open(sess_pkl_dir, 'wb') as file:
                                    cloudpickle.dump(data, file)
                            except:
                                pass
                    except:
                        print(date, fly, sess)
                        pass



dict_keys(['ref_ch1', 'ref_ch1_maxp', 'ref_ch2', 'ref_ch2_maxp', 'inner_ring', 'outer_ring_EB', 'outer_ring_R4d', 'background', 'rois_EB', 'rois_R4d', 'n_ch', 'ref_img'])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
06_08_2023 fly1 baseline-004


In [9]:
genotype = "ER4d_sytjGCaMP7f_EPG_jRGECO1a"
basedir = pathlib.Path(f'/media/mplitt/SSD_storage1/2PData/{genotype}')

for date_dir in basedir.glob('*'):
    date = date_dir.stem
    for fly_dir in date_dir.glob('fly*'): 
        fly = fly_dir.stem
        
        for sess_dir in fly_dir.glob("*"):
            if sess_dir.is_dir() and "SingleImage" not in sess_dir.stem:
                sess = sess_dir.stem
                
                sess_pkl_dir = f'/media/mplitt/SSD_storage/2P_scratch/{genotype}/{date}/{fly}/{sess}/data.pkl'
                if os.path.exists(sess_pkl_dir):
                    print(date, fly, sess)
                    with open(sess_pkl_dir, 'rb') as file:
                        data = cloudpickle.load(file)
                           
    
                    F_EB_bcorr, F_R4d_bcorr = data['F_EB_bcorr'], data['F_R4d_bcorr']
                    F_EB_bleed = 0*F_EB_bcorr
                    F_R4d_bleed = 0*F_R4d_bcorr
                    lr = LinReg().fit(F_EB_bcorr[1,:, :].T, F_EB_bcorr[0,:,:].T)
                    F_EB_bleed[0,:,:] = F_EB_bcorr[0,:,:]-1.*lr.predict(F_EB_bcorr[1,:,:].T).T 

                    lr = LinReg().fit(F_R4d_bcorr[0,:, :].T, F_R4d_bcorr[1,:,:].T)
                    F_R4d_bleed[1,:,:] = F_R4d_bcorr[1,:,:]-1.*lr.predict(F_R4d_bcorr[0,:,:].T).T 
                    
                    data['F_EB_bleed']=F_EB_bleed
                    data['F_R4d_bleed']=F_R4d_bleed
                    
                    
                    F_EB_sm = sp.ndimage.gaussian_filter1d(F_EB_bleed,2,axis=-1)
                    F_EB_sm = sp.ndimage.gaussian_filter1d(F_EB_sm,1.,axis=1, mode='wrap')
                    
                    data['F_EB_sm']=F_EB_sm
    
                    F_R4d_sm = sp.ndimage.gaussian_filter1d(F_R4d_bleed,2,axis=-1)
                    F_R4d_sm = sp.ndimage.gaussian_filter1d(F_R4d_sm,1.,axis=1, mode='wrap')
            
                    data['F_R4d_sm']=F_R4d_sm

                    dff_R4d = sp.stats.zscore(F_R4d_sm[1,:,:],axis=-1)#/np.linalg.norm(F_R4d_sm[1,:,:], axis=0, keepdims=True)
                    dff_EB = sp.stats.zscore(F_EB_sm[0,:,:],axis=-1)#/np.linalg.norm(F_EB_sm[0,:,:], axis=0, keepdims=True)
                    
                    data['dff_EB']=dff_EB
                    data['dff_R4d']=dff_R4d

                    heading = data['heading']
                    

                    bin_centers =  np.linspace(-np.pi, np.pi, num=16)[:,np.newaxis]
                    dff_EB_complex = dff_EB*np.cos(bin_centers) + 1j*dff_EB*np.sin(bin_centers)
                    rho_EB, phi_EB = np.abs(dff_EB_complex.mean(axis=0)), np.angle(dff_EB_complex.mean(axis=0))

                    data['dff_EB_complex']=dff_EB_complex
                    data['rho_EB']=rho_EB
                    data['phi_EB']=phi_EB
                    
                    dff_R4d_complex = dff_R4d*np.cos(bin_centers) + 1j*dff_R4d*np.sin(bin_centers)
                    rho_R4d, phi_R4d = np.abs(dff_R4d_complex.mean(axis=0)), np.angle(dff_R4d_complex.mean(axis=0))
                    
                    data['dff_R4d_complex']=dff_R4d_complex
                    data['rho_R4d']=rho_R4d
                    data['phi_R4d']=phi_R4d
                    
                    with open(sess_pkl_dir,'wb') as file:
                        cloudpickle.dump(data,file)

11_08_2023 fly2 baseline-000
11_08_2023 fly2 open_loop-001
11_08_2023 fly1 baseline-000
11_08_2023 fly3 baseline-000
11_08_2023 fly3 open_loop-002
10_10_2023 fly1 baseline-000
10_10_2023 fly1 baseline-001
30_10_2023 fly2 closed_loop-000
30_10_2023 fly2 closed_loop-001
30_10_2023 fly1 closed_loop-000
30_10_2023 fly1 closed_loop-003
30_10_2023 fly1 closed_loop-001
30_10_2023 fly3 closed_loop-000
06_11_2023 fly1 closed_loop-002
06_11_2023 fly1 closed_loop-000
06_08_2023 fly2 baseline-001
07_08_2023 fly1 baseline-000
07_08_2023 fly3 baseline-000
07_08_2023 fly3 baseline-001
12_10_2023 fly2 baseline-000
12_10_2023 fly2 baseline-001
12_10_2023 fly2 baseline_to_dark-002
12_10_2023 fly3 baseline-000
12_10_2023 fly3 baseline-001
11_10_2023 fly6 baseline-001
11_10_2023 fly5 baseline-000
11_10_2023 fly2 baseline-000
11_10_2023 fly2 baseline-001
11_10_2023 fly7 baseline-000
11_10_2023 fly7 baseline-001
11_10_2023 fly7 baseline_to_dark-002
11_10_2023 fly4 baseline_open_loop-002
31_10_2023 fly5 clos